In [1]:
#!pip install seaborn
#!pip install openpyxl
#!pip install sklearn



In [2]:
import pandas as pd
import numpy as np
import random as rd
import time
import csv
import seaborn as sbs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [3]:
file_data = 'SN_m_tot_V2.0.csv'
path_name='../datasets/'
path_name_results='../results/'
file_result = 'Result_MLP_sunspot.csv'

In [4]:
dataset = pd.read_csv(f'{path_name}{file_data}', sep =';', encoding = 'latin1', decimal='.')
dataset.columns = ['year','month', 'date','total_sunspot_number','std_derivation','num_observations','def_prov_indicator']

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3295 entries, 0 to 3294
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  3295 non-null   int64  
 1   month                 3295 non-null   int64  
 2   date                  3295 non-null   float64
 3   total_sunspot_number  3295 non-null   float64
 4   std_derivation        3295 non-null   float64
 5   num_observations      3295 non-null   int64  
 6   def_prov_indicator    3295 non-null   int64  
dtypes: float64(3), int64(4)
memory usage: 180.3 KB


In [6]:
#checks if there are null variables
dataset.isna().sum()

year                    0
month                   0
date                    0
total_sunspot_number    0
std_derivation          0
num_observations        0
def_prov_indicator      0
dtype: int64

In [7]:
def salvar_resultado(nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration):
  #Script to write training cycle results
  data = [nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration]
  fields = ['Dataset','Best Params','n_time_steps','MSE', 'RMSE', 'MAE', 'MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['Dataset','Best Params','n_time_steps','MSE', 'RMSE', 'MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)    

In [8]:
def previsao_MLP(nm_dataset, dataset, n_time_steps):
  # dataframe tratament
  df = pd.DataFrame()
  df['num_observations']=dataset['num_observations']  

  # time serire transform - shit 1 step time

  for n_step in range(1,n_time_steps+1,1):
    df['vl-'+str(n_step)]=dataset['num_observations'].shift(n_step)  
      
  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  80% treinam  20% test
  nlinhas = int(np.round(df.shape[0] *0.80)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1: 1 + n_time_steps]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1: 1 + n_time_steps] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  # Cross-validated for time series
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  param_grid = {
      'hidden_layer_sizes': [(4,6,1),(2,6,1),(6,12,1),(6,18,1)], # MLP layers
      'max_iter': [ 500], # maximum iterations
      'activation': [ 'relu','identity'], # activation function
      'solver': ['adam'], # weight optimization algorithm
      'alpha': [0.0001, 0.001, 0.01],  # alpha strength of regularization
  }
  
  modelo = MLPRegressor(random_state=0)
  
  grid = GridSearchCV(modelo, param_grid, n_jobs= -1,scoring='neg_mean_absolute_percentage_error', cv=ts_cv, verbose=1)
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))  

  # Stores the training execution end time
  Hora_Fim = time.time()  

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio  

  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)   

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))   

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test) 
  
  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100   
  
  sMAPE = round(
		np.mean(
			np.abs(predict - y_test) /
			((np.abs(predict) + np.abs(y_test)))
		)*100, 2
	)

  salvar_resultado(nm_dataset, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duracao)

In [9]:
#create file to results
criar_arquivo_resultado()

print('forecast for new sunspot number')
for n_time_steps in range(1,13): #predict with 1 to 13 past values of medition
    grid = previsao_MLP('sunspot', dataset, n_time_steps)

forecast for new sunspot number
Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['sunspot', "{'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (4, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 1, 362714.4474073092, 602.2577914874238, 512.3202850861192, 95.03598814735584, 91.4, 85.21172618865967]
Fitting 5 folds for each of 24 candidates, totalling 120 fits
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['sunspot', "{'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (6, 12, 1), 'max_iter': 500, 'solver': 'adam'}", 2, 6869.391492407489, 82.88179228520272, 33.470482597623004, 9.152552958759049, 4.61, 51.81887125968933]
Fitting 5 folds for each of 24 candidates, totalling 120 fits
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['sunspot', "{'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (6, 18, 1), 'max_iter': 500, 'solver': 'adam'}", 3, 7236